In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')

In [2]:
import sys
import os
sys.path.append(os.path.join(os.path.curdir, ".."))

In [3]:
from settings import ELASTICSEARCH_CONNECTIONS
from elasticsearch_dsl.connections import connections
connections.configure(timeout=120, **ELASTICSEARCH_CONNECTIONS)

In [4]:
from name_utils import parse_fullname

with open("../test_names.txt", "r") as fp:
    names = list(fp)

# names = [
#     "Блабласюк Дмитро Романович",
#     "РОМАНЮК Ярослав Михайлович",
#     "Валентин Петрович БАРБАРА",
# ]

names = list(map(str.strip, names))
names = list(map(parse_fullname, names))


candidates = [
    "Дмитрий Романович Блабласюк",
    "Сергій Насалик",
    "BARBARA Valentyn Petrovych",
]

candidates = list(map(str.strip, candidates))
candidates = list(map(parse_fullname, candidates))

In [5]:
from models.names import NameVariant

matched_names = []
for x in range(100):
    matched_names += NameVariant.batch_request(names[x * 20:(x + 1) * 20])

# matched_names = NameVariant.batch_request(names)
matched_candidates = NameVariant.batch_request(candidates)

In [6]:
from matcher import Matcher

m = Matcher(dict(enumerate(matched_names)))

In [7]:
for x in matched_candidates:
    print(m.match(x))

None
None
[1]


In [8]:
len(m.seed)

52526

In [5]:
from hasher import batch_request

In [10]:
print(len(names))

%timeit res = batch_request(names[:1000])
%timeit res = batch_request(names[:2000])
%timeit -n 5 res = batch_request(names)
%timeit -n 5 res = batch_request(names + names + names + names + names)

3561
10 loops, best of 3: 136 ms per loop
1 loop, best of 3: 268 ms per loop
5 loops, best of 3: 516 ms per loop
5 loops, best of 3: 2.99 s per loop


In [6]:
res = batch_request(names[:1000])

In [7]:
res[0]

(({'label': 'l', 'lemma': 24467, 'term': 'Романюк'},
  {'label': 'l', 'lemma': 160203, 'term': 'Романюк'}),
 ({'label': 'f', 'lemma': 6709, 'term': 'Ярослав'},
  {'label': 'l', 'lemma': 27858, 'term': 'Ярослав'},
  {'label': 'f', 'lemma': 6710, 'term': 'Ярослав'},
  {'label': 'l', 'lemma': 27857, 'term': 'Ярослав'}),
 ({'label': 'l', 'lemma': 13292, 'term': 'Михайлович'},
  {'label': 'f', 'lemma': 13297, 'term': 'Михайлович'},
  {'label': 'l', 'lemma': 13290, 'term': 'Михайлович'},
  {'label': 'l', 'lemma': 13296, 'term': 'Михайлович'},
  {'label': 'f', 'lemma': 13300, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13302, 'term': 'Михайлович'},
  {'label': 'f', 'lemma': 1358, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13295, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13289, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13303, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13298, 'term': 'Михайлович'},
  {'label': 'p', 'lemma': 13293, 'term': 'Михайлович'},
  {'label'

In [23]:
from matcher import Matcher

# m = Matcher(dict(enumerate(res[:1])))

for x in m.generate(res[0]):
    print(x)

defaultdict(<class 'list'>, {'f': [(1, {'lemma': 6709, 'label': 'f', 'term': 'Ярослав'}), (1, {'lemma': 6710, 'label': 'f', 'term': 'Ярослав'}), (2, {'lemma': 13297, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 13300, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 1358, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 13301, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 13288, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 13291, 'label': 'f', 'term': 'Михайлович'}), (2, {'lemma': 1357, 'label': 'f', 'term': 'Михайлович'})], 'l': [(0, {'lemma': 24467, 'label': 'l', 'term': 'Романюк'}), (0, {'lemma': 160203, 'label': 'l', 'term': 'Романюк'}), (1, {'lemma': 27858, 'label': 'l', 'term': 'Ярослав'}), (1, {'lemma': 27857, 'label': 'l', 'term': 'Ярослав'}), (2, {'lemma': 13292, 'label': 'l', 'term': 'Михайлович'}), (2, {'lemma': 13290, 'label': 'l', 'term': 'Михайлович'}), (2, {'lemma': 13296, 'label': 'l', 'term': 'Михайлович'}), (2, {'lemma': 13294, 'label': 'l', 

In [17]:
m = Matcher(dict(enumerate(res)))

candidates = [
    "Дмитрий Романович Блабласюк",
    "Сергій Насалик",
    "РОМАНЮКу Я М",
    "BARBARA V. P.",
]

candidates = list(map(str.strip, candidates))
candidates = list(map(parse_fullname, candidates))

matched_candidates = batch_request(candidates)

for x in matched_candidates:
    print(m.match(x))

None
None
[0]
[1]
